In [6]:
import os, sys, random, json, copy
from datetime import datetime, timezone

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
)
from sklearn.metrics import mean_absolute_error, mean_squared_error

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# set deterministic seeds
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# pick device
device = (
    torch.device("cuda")
    if torch.cuda.is_available() else
    torch.device("mps")
    if torch.backends.mps.is_available() else
    torch.device("cpu")
)
print(f"> Using device: {device}")

> Using device: cuda


## 1. Data Loading and Feature Engineering

In [7]:
# Load the data
df = pd.read_csv("df_all.csv", index_col="startTime", parse_dates=True)

# Crop the DataFrame to the specified date range
df = df.loc["2021-07-01":"2025-06-30"]

# Drop unnecessary columns
df = df.drop(columns=[
    'Forecast Wind',
    'Forecast Solar',
    'Actual Wind',
    'Actual Solar',
    'Settlement Period',
])

# Calculate time of day features
minutes = df.index.hour * 60 + df.index.minute
frac_day = minutes / (24 * 60)
df['tod_sin'] = np.sin(2 * np.pi * frac_day)
df['tod_cos'] = np.cos(2 * np.pi * frac_day)

# Calculate day of week features
day_of_week = df.index.dayofweek
frac_week = day_of_week / 7
df['dow_sin'] = np.sin(2 * np.pi * frac_week)
df['dow_cos'] = np.cos(2 * np.pi * frac_week)

# Calculate month of year features
month = df.index.month
frac_year = (month - 1) / 12
df['moy_sin'] = np.sin(2 * np.pi * frac_year)
df['moy_cos'] = np.cos(2 * np.pi * frac_year)


# — splits
train_end = '2025-03-01'  # start of validation
val_end   = '2025-05-01'  # start of test

# slice once …
train_df = df.loc[:train_end]
val_df   = df.loc[train_end:val_end]
test_df  = df.loc[val_end:]

# … then unpack X & y in one go without repeating .drop
X_train, y_train = train_df.drop(columns=['Imbalance Price']), train_df['Imbalance Price']
X_val,      y_val      = val_df.drop(columns=['Imbalance Price']),    val_df['Imbalance Price']
X_test,     y_test     = test_df.drop(columns=['Imbalance Price']),   test_df['Imbalance Price']


# lists of feature‐columns
time_feats  = [c for c in ['tod_sin','tod_cos','dow_sin','dow_cos','moy_sin','moy_cos'] if c in df]
other_feats = [c for c in X_train.columns if c not in time_feats]

## 2. Models, Dataset & Factories

In [8]:
# ──────────── a. Dataset Definition ─────────────────────────────────

class LSTMDataset(Dataset):
    def __init__(self, X, y, seq_len, horizon=1):
        self.X = torch.as_tensor(X, dtype=torch.float32)
        self.y = torch.as_tensor(y, dtype=torch.float32)
        self.seq_len = seq_len
        self.horizon = horizon

    def __len__(self):
        return self.X.shape[0] - self.seq_len - self.horizon + 1

    def __getitem__(self, idx):
        x_seq    = self.X[idx : idx + self.seq_len]
        target_i = idx + self.seq_len - 1 + self.horizon
        y_target = self.y[target_i]
        return x_seq, y_target


# ──────────── b. Layer Definitions ──────────────────────────────────

class SeasonalAttn(nn.Module):
    def __init__(self, seq_len=48):
        super().__init__()
        self.attn = nn.Parameter(torch.empty(seq_len))
        nn.init.uniform_(self.attn, -0.01, 0.01)

    def forward(self, x):
        return x * self.attn.view(1, -1, 1) # (B,N,F)


class BiLSTM(nn.Module):
    def __init__(self, num_feats, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=num_feats,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
        )

    def forward(self, x):
        out, (_ , _) = self.lstm(x)
        return out


class AttentionPool(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.W_h = nn.Linear(input_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, x):
        e = self.v(torch.tanh(self.W_h(x)))   # (B, N, 1)
        alpha = torch.softmax(e, dim=1)       # (B, N, 1)
        context = torch.sum(alpha * x, dim=1) # (B, 2H)
        return context


# ──────────── c. Model Definitions ──────────────────────────────────

class SA_BiLSTM(nn.Module):
    def __init__(self, num_feats, seq_len=48, hidden_size=64, num_layers=2):
        super().__init__()
        self.seasonal = SeasonalAttn(seq_len)
        self.lstm = BiLSTM(num_feats, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        x = self.seasonal(x)       # (B, N, F)
        out = self.lstm(x)         # (B, N, 2H)
        last = out[:, -1, :]       # (B, 2H)
        return self.fc(last).squeeze(-1)  # (B,)


class SA_BiLSTM_AttnPool(nn.Module):
    def __init__(self, num_feats, seq_len=48, hidden_size=64, num_layers=2):
        super().__init__()
        self.seasonal = SeasonalAttn(seq_len)
        self.lstm = BiLSTM(num_feats, hidden_size, num_layers)
        self.pool = AttentionPool(input_dim=hidden_size * 2, hidden_dim=hidden_size)
        self.fc = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        x = self.seasonal(x)       # (B, N, F)
        out = self.lstm(x)         # (B, N, 2H)
        context = self.pool(out)   # (B, 2H)
        return self.fc(context).squeeze(-1)  # (B,)

TRANSFORMER_FACTORY = {
    "MinMax":   MinMaxScaler,
    "Standard": StandardScaler,
    "Robust":   RobustScaler,
    "MaxAbs":   MaxAbsScaler
}
MODEL_FACTORY = {
    "SA_BiLSTM":             SA_BiLSTM,
    "SA_BiLSTM_AttnPool":    SA_BiLSTM_AttnPool
}
LOSS_FACTORY = {
    "MSE":    nn.MSELoss,
    "MAE":    nn.L1Loss,
    "Huber":  nn.SmoothL1Loss
}


## 3. Search Stage 1 -- Big 3

In [9]:
def objective_stage1(trial):
    # ── 1) Sample hyper‐params ────────────────────────────────────────
    lr          = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    batch_size  = trial.suggest_categorical("batch_size", [64, 128, 256])

    # ── Fixed settings ────────────────────────────────────────────────
    seq_len, horizon, num_layers = 48, 1, 2
    model_name, scaler_used, loss_used = "SA_BiLSTM", "MaxAbs", "Huber"
    beta, max_epochs, patience = 0.01, 20, 10  # shorter for speed

    # report start
    print(f"\n→ Trial {trial.number}: lr={lr:.2e}, hidden={hidden_size}, bs={batch_size}")

    # ── 2) Data prep ──────────────────────────────────────────────────
    transformer = ColumnTransformer(
        [("scale", TRANSFORMER_FACTORY[scaler_used](), other_feats)],
        remainder="passthrough", verbose_feature_names_out=False
    )
    X_tr = transformer.fit_transform(X_train)
    X_va = transformer.transform(X_val)

    scaler_y = TRANSFORMER_FACTORY[scaler_used]()\
                   .fit(y_train.values.reshape(-1,1))
    y_tr = scaler_y.transform(y_train.values.reshape(-1,1)).ravel()
    y_va = scaler_y.transform(y_val.values.reshape(-1,1)).ravel()

    tr_lo = DataLoader(
        LSTMDataset(X_tr, y_tr, seq_len, horizon),
        batch_size=batch_size, shuffle=True,  pin_memory=True
    )
    va_lo = DataLoader(
        LSTMDataset(X_va, y_va, seq_len, horizon),
        batch_size=batch_size, shuffle=False, pin_memory=True
    )

    # ── 3) Model / Optimizer / Loss / Scheduler ───────────────────────
    model     = MODEL_FACTORY[model_name](
                    num_feats=X_tr.shape[1],
                    seq_len=seq_len,
                    hidden_size=hidden_size,
                    num_layers=num_layers
                ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = (LOSS_FACTORY[loss_used](beta=beta)
                 if loss_used=="Huber"
                 else LOSS_FACTORY[loss_used]())
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #                 optimizer, mode="min", factor=0.5,
    #                 patience=5, min_lr=1e-7
    #             )

    best_val = float("inf")
    epochs_no_improve = 0

    # ── 4) Training w/ pruning & prints ───────────────────────────────
    for epoch in range(1, max_epochs+1):
        # — train —
        model.train()
        tr_loss = 0.0
        for xb, yb in tr_lo:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb).squeeze(-1), yb)
            loss.backward()
            optimizer.step()
            tr_loss += loss.item()
        tr_loss /= len(tr_lo)

        # — validate —
        model.eval()
        va_loss = 0.0
        with torch.no_grad():
            for xb, yb in va_lo:
                xb, yb = xb.to(device), yb.to(device)
                va_loss += criterion(model(xb).squeeze(-1), yb).item()
        va_loss /= len(va_lo)

        # — scheduler & LR print —
        # scheduler.step(va_loss)
        # current_lr = scheduler.get_last_lr()[0]
        print(
            f"[Trial {trial.number}] "
            f"Epoch {epoch:02d}/{max_epochs}  "
            # f"lr={current_lr:.2e}  "
            f"train={tr_loss:.4f}  val={va_loss:.4f}"
        )

        # — report & prune —
        trial.report(va_loss, epoch)
        if trial.should_prune():
            print(f"→ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        # — manual early stopping —
        if va_loss < best_val:
            best_val = va_loss
            epochs_no_improve = 0
            best_weights = copy.deepcopy(model.state_dict())
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"→ Trial {trial.number} early-stopped at epoch {epoch}")
                break

    # ── 5) Load best & compute final metrics on validation set ─────────
    model.load_state_dict(best_weights)
    model.eval()
    scaled_preds, scaled_trues = [], []
    with torch.no_grad():
        for xb, yb in va_lo:
            xb = xb.to(device)
            p  = model(xb).squeeze(-1).cpu().numpy()
            scaled_preds.append(p)
            scaled_trues.append(yb.numpy())
    scaled_preds = np.concatenate(scaled_preds)
    scaled_trues = np.concatenate(scaled_trues)

    # invert scaling
    preds = scaler_y.inverse_transform(scaled_preds.reshape(-1,1)).flatten()
    trues = scaler_y.inverse_transform(scaled_trues.reshape(-1,1)).flatten()
    err   = trues - preds

    mae   = mean_absolute_error(trues, preds)
    rmse  = np.sqrt(mean_squared_error(trues, preds))
    smape = np.mean(2*np.abs(err)/(np.abs(trues)+np.abs(preds)+1e-8))*100

    print(
        f"→ Trial {trial.number} finished: "
        f"best_val={best_val:.4f}  "
        f"MAE={mae:.4f}  RMSE={rmse:.4f}  sMAPE={smape:.2f}%"
    )

    # store metrics
    trial.set_user_attr("val_mae", mae)
    trial.set_user_attr("val_rmse", rmse)
    trial.set_user_attr("val_smape", smape)

    return best_val


# ── Run Stage 1 ───────────────────────────────────────────────────────
study1 = optuna.create_study(
    direction="minimize",
    sampler=TPESampler(),
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5)
)
study1.optimize(objective_stage1, n_trials=30)
print("✔ Stage 1 done:", study1.best_params, "val_loss=", study1.best_value)


[I 2025-07-29 15:18:17,995] A new study created in memory with name: no-name-47312671-c8ac-4f4b-b4a0-e1725b68b9e1
/tmp/ipython-input-9-2069821114.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr          = trial.suggest_loguniform("lr", 1e-5, 1e-3)



→ Trial 0: lr=1.44e-04, hidden=128, bs=128
[Trial 0] Epoch 01/20  train=0.0127  val=0.0033
[Trial 0] Epoch 02/20  train=0.0086  val=0.0034
[Trial 0] Epoch 03/20  train=0.0073  val=0.0025
[Trial 0] Epoch 04/20  train=0.0068  val=0.0027
[Trial 0] Epoch 05/20  train=0.0063  val=0.0024
[Trial 0] Epoch 06/20  train=0.0061  val=0.0023
[Trial 0] Epoch 07/20  train=0.0059  val=0.0023
[Trial 0] Epoch 08/20  train=0.0059  val=0.0020
[Trial 0] Epoch 09/20  train=0.0060  val=0.0020
[Trial 0] Epoch 10/20  train=0.0058  val=0.0024
[Trial 0] Epoch 11/20  train=0.0059  val=0.0023
[Trial 0] Epoch 12/20  train=0.0057  val=0.0022
[Trial 0] Epoch 13/20  train=0.0057  val=0.0019
[Trial 0] Epoch 14/20  train=0.0057  val=0.0023
[Trial 0] Epoch 15/20  train=0.0058  val=0.0019
[Trial 0] Epoch 16/20  train=0.0057  val=0.0022
[Trial 0] Epoch 17/20  train=0.0057  val=0.0023
[Trial 0] Epoch 18/20  train=0.0057  val=0.0018
[Trial 0] Epoch 19/20  train=0.0056  val=0.0018


[I 2025-07-29 15:20:32,848] Trial 0 finished with value: 0.0018062540279377413 and parameters: {'lr': 0.00014391731750860453, 'hidden_size': 128, 'batch_size': 128}. Best is trial 0 with value: 0.0018062540279377413.


[Trial 0] Epoch 20/20  train=0.0057  val=0.0018
→ Trial 0 finished: best_val=0.0018  MAE=19.3820  RMSE=24.9520  sMAPE=32.10%

→ Trial 1: lr=1.04e-04, hidden=64, bs=128


/tmp/ipython-input-9-2069821114.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr          = trial.suggest_loguniform("lr", 1e-5, 1e-3)


...

[Trial 28] Epoch 20/20  train=0.0051  val=0.0018
→ Trial 28 finished: best_val=0.0017  MAE=18.8516  RMSE=24.4400  sMAPE=31.40%

→ Trial 29: lr=3.44e-04, hidden=32, bs=64


/tmp/ipython-input-9-2069821114.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr          = trial.suggest_loguniform("lr", 1e-5, 1e-3)


[Trial 29] Epoch 01/20  train=0.0112  val=0.0041
[Trial 29] Epoch 02/20  train=0.0068  val=0.0025
[Trial 29] Epoch 03/20  train=0.0060  val=0.0023
[Trial 29] Epoch 04/20  train=0.0058  val=0.0019
[Trial 29] Epoch 05/20  train=0.0058  val=0.0022
[Trial 29] Epoch 06/20  train=0.0057  val=0.0019
[Trial 29] Epoch 07/20  train=0.0057  val=0.0021
[Trial 29] Epoch 08/20  train=0.0057  val=0.0018
[Trial 29] Epoch 09/20  train=0.0057  val=0.0018
[Trial 29] Epoch 10/20  train=0.0056  val=0.0019
[Trial 29] Epoch 11/20  train=0.0056  val=0.0019
[Trial 29] Epoch 12/20  train=0.0056  val=0.0021
[Trial 29] Epoch 13/20  train=0.0056  val=0.0022
[Trial 29] Epoch 14/20  train=0.0056  val=0.0019
[Trial 29] Epoch 15/20  train=0.0056  val=0.0019
[Trial 29] Epoch 16/20  train=0.0056  val=0.0018
[Trial 29] Epoch 17/20  train=0.0056  val=0.0019
[Trial 29] Epoch 18/20  train=0.0055  val=0.0020
[Trial 29] Epoch 19/20  train=0.0055  val=0.0019


[I 2025-07-29 15:57:41,658] Trial 29 finished with value: 0.0017878655850401392 and parameters: {'lr': 0.00034437161892647236, 'hidden_size': 32, 'batch_size': 64}. Best is trial 12 with value: 0.0017215024892483716.


[Trial 29] Epoch 20/20  train=0.0055  val=0.0018
→ Trial 29 finished: best_val=0.0018  MAE=19.3614  RMSE=24.7975  sMAPE=32.09%
✔ Stage 1 done: {'lr': 0.0009637770438198408, 'hidden_size': 64, 'batch_size': 128} val_loss= 0.0017215024892483716
